In [11]:
import tensorflow as tf
from tensorflow import keras
from tf_quantization.transforms.quantize_model import quantize_model

from datetime import datetime
from packaging import version


In [8]:
# Load imagenet 100k dataset
import imagenet_mini

tr_ds = imagenet_mini.get_imagenet_mini_dataset(split="train")
tr_ds = tr_ds.map(imagenet_mini.get_preprocess_image_fn(image_size=(224, 224)))

train_ds = tr_ds \
    .map(lambda data: (data['image'], data['label'])) \
    .batch(128)

ds = imagenet_mini.get_imagenet_mini_dataset(split="val")
ds = ds.map(imagenet_mini.get_preprocess_image_fn(image_size=(224, 224)))

test_ds = ds.map(lambda data: (data['image'], data['label'])).batch(64)

2023-02-27 13:01:53.127448: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


In [2]:
model = tf.keras.applications.MobileNet(weights='imagenet', input_shape=(224, 224, 3), alpha=0.25)

Metal device set to: Apple M2 Pro


2023-02-27 13:00:16.007261: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-27 13:00:16.007304: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
model.summary()

Model: "mobilenet_0.25_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 8)       216       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 8)      32        
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 8)       0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 8)      72        
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 8)      32        
 ation)                                         

In [4]:
print("Quantize model")

bit_8_conf = {"weight_bits": 8, "activation_bits": 8}
bit_7_conf = {"weight_bits": 7, "activation_bits": 8}
bit_6_conf = {"weight_bits": 6, "activation_bits": 8}
bit_5_conf = {"weight_bits": 5, "activation_bits": 8}
bit_4_conf = {"weight_bits": 4, "activation_bits": 8}
bit_3_conf = {"weight_bits": 3, "activation_bits": 8}
bit_2_conf = {"weight_bits": 2, "activation_bits": 8}

q_aware_model = quantize_model(model, [
    bit_8_conf, bit_8_conf, bit_8_conf, bit_8_conf, bit_8_conf,
    bit_8_conf, bit_8_conf, bit_8_conf, bit_8_conf, bit_8_conf,
    bit_8_conf, bit_8_conf, bit_8_conf, bit_8_conf, bit_8_conf,
    bit_8_conf, bit_8_conf, bit_8_conf, bit_8_conf, bit_8_conf,
    bit_8_conf, bit_8_conf, bit_8_conf, bit_8_conf, bit_8_conf,
    bit_8_conf, bit_8_conf, bit_8_conf, bit_8_conf, bit_8_conf,
    bit_8_conf, bit_8_conf, bit_8_conf, bit_8_conf, bit_8_conf,
    bit_8_conf, bit_8_conf
])

q_aware_model.summary()

Quantize model
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Model: "mobilenet_0.25_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 quantize_layer (QuantizeLay  (None, 224, 224, 3)      3         
 er)                                                             
                                                                 
 quant_conv1 (QuantizeWrappe  (None, 112, 112, 8)      233       
 rV2)                                                            
                                                                 
 quant_conv1_bn (QuantizeWra  (None, 112, 112, 8)      33        
 pperV2)                      

In [5]:
# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.01),
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                      metrics=['accuracy'])

In [6]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [7]:
import tensorboard
tensorboard.__version__

'2.11.2'

In [12]:
# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [13]:
# Train for one epoch
q_aware_model.fit(train_ds, epochs=1, validation_data=test_ds, callbacks=[tensorboard_callback], initial_epoch=0)

2023-02-27 13:06:27.860953: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-27 13:06:30.107363: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - ETA: 0s - loss: 4.3563 - accuracy: 0.1649

2023-02-27 13:14:00.954472: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 462s 578ms/step - loss: 4.3563 - accuracy: 0.1649 - val_loss: 9.9378 - val_accuracy: 0.0115


In [15]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 94723), started 0:08:01 ago. (Use '!kill 94723' to kill it.)